In [66]:
import pandas as pd
import numpy as np
import json
from os import makedirs, listdir
from os.path import join, exists
import re

In [70]:
ARTICLES_DIR = 'data/guardian_articles'
JOINED_ARTICLES_FILE = 'data/guardian_combined.csv'
df = None
for filename in listdir(ARTICLES_DIR):
    if filename.endswith('.json'):
        filepath = join(articles_dir, filename)
        # join all json file in a dataframe
        df_current = pd.read_json(filepath, encoding='utf-8')
        df = pd.concat(objs=[df,df_current], axis=0, ignore_index=True)
        
    

In [71]:
df.head()

,apiUrl,fields,id,isHosted,pillarId,pillarName,sectionId,sectionName,type,webPublicationDate,webTitle,webUrl
0,https://content.guardianapis.com/sport/2017/no...,{'headline': 'ATP Finals: Goffin shocks Federe...,sport/2017/nov/18/roger-federer-,False,pillar/sport,Sport,sport,Sport,article,2017-11-18T23:27:54Z,ATP Finals: Goffin shocks Federer to set up ti...,https://www.theguardian.com/sport/2017/nov/18/...
1,https://content.guardianapis.com/football/2017...,{'headline': 'Bloody nose for Sergio Ramos as ...,football/2017/nov/18/atletico-madrid-real-madr...,False,pillar/sport,Sport,football,Football,article,2017-11-18T22:35:33Z,Bloody nose for Sergio Ramos as Real and Atlét...,https://www.theguardian.com/football/2017/nov/...
2,https://content.guardianapis.com/football/2017...,{'headline': 'David Moyes insists West Ham is ...,football/2017/nov/18/west-ham-david-moyes,False,pillar/sport,Sport,football,Football,article,2017-11-18T22:30:43Z,David Moyes insists West Ham is not his last c...,https://www.theguardian.com/football/2017/nov/...
3,https://content.guardianapis.com/sport/2017/no...,{'headline': 'Ashes 2017-18: player-by-player ...,sport/2017/nov/18/australia-england-first-test...,False,pillar/sport,Sport,sport,Sport,article,2017-11-18T22:15:09Z,Ashes 2017-18: player-by-player guide to the A...,https://www.theguardian.com/sport/2017/nov/18/...
4,https://content.guardianapis.com/politics/2017...,{'headline': 'Gerry Adams steps down as Sinn F...,politics/2017/nov/18/gerry-adams-steps-down-as...,False,pillar/news,News,politics,Politics,article,2017-11-18T22:10:48Z,Gerry Adams steps down as Sinn Féin president,https://www.theguardian.com/politics/2017/nov/...


In [72]:
df.columns

Index(['apiUrl', 'fields', 'id', 'isHosted', 'pillarId', 'pillarName',
       'sectionId', 'sectionName', 'type', 'webPublicationDate', 'webTitle',
       'webUrl'],
      dtype='object')

In [73]:
# for all articles extract their text which is under fileds -> bodytext
# df.iloc[0].fields -> show contents of a fields columns
body_text = []
headlines = []
wordcount = []
for index in range(df.shape[0]):
    text = df.iloc[index].fields['bodyText']
    headline = df.iloc[index].fields['headline']
    count = df.iloc[index].fields['wordcount']
    body_text.append(text)
    headlines.append(headline)
    wordcount.append(count)

df['bodyText'] = body_text
df['headlines'] = headlines
df['wordcount'] = wordcount

# Remove articles where body is empty, also selects articles from US


In [74]:
df = df[df.bodyText != ""]


In [75]:
#df.sectionName[0:100]
section_mask = ((df.sectionName == 'US news') | (df.sectionName == 'Sport') 
                |  (df.sectionName == 'Business') | (df.sectionName == 'politics') | (df.sectionName == 'World news')
                | (df.sectionName == 'Film') | (df.sectionName == 'Fashion') 
               )


In [76]:
print(df.shape)
df = df[section_mask]
print(df.shape)


(67281, 15)
(19143, 15)


In [77]:
#replace new line with "" , find other patterns later
for idex, item in enumerate(df['bodyText']):
    df['bodyText'][index] = re.sub("(\n)", "", item)

for idex, item in enumerate(df['headlines']):
    df['headlines'][index] = re.sub("(\n)", "", item)
    
#saved this data frame to csv file

In [78]:
df.to_csv(JOINED_ARTICLES_FILE, sep=',', encoding='utf-8')

In [79]:
df_saved = pd.read_csv(JOINED_ARTICLES_FILE, sep=',', encoding='utf-8')

In [85]:
col_selected = ['webUrl', 'sectionName', 'type', 'bodyText', 'headlines', 'wordcount']
df_saved = df_saved[col_selected]

In [86]:
df_saved.head()

,webUrl,sectionName,type,bodyText,headlines,wordcount
0,https://www.theguardian.com/sport/2017/nov/18/...,Sport,article,Roger Federer leaves his magnificent season on...,ATP Finals: Goffin shocks Federer to set up ti...,907
1,https://www.theguardian.com/sport/2017/nov/18/...,Sport,article,DAVID WARNER Bullies bowlers and loves beating...,Ashes 2017-18: player-by-player guide to the A...,741
2,https://www.theguardian.com/world/2017/nov/18/...,World news,article,More than a year after the UK government pledg...,First child refugee from Greek camps comes to UK,914
3,https://www.theguardian.com/world/2017/nov/18/...,World news,article,Scientists have warned there could be a big in...,Upsurge in big earthquakes predicted for 2018 ...,510
4,https://www.theguardian.com/film/2017/nov/18/h...,Film,article,The Observer has gained access to a secret hit...,Harvey Weinstein had secret hitlist of names t...,938


In [83]:
# df_kaggle = pd.read_csv('test_kaggle.csv', sep =',')
# df_kaggle.head()